In [ ]:
from PIL import Image
import numpy as np
from os import path

def image_to_np_array(image_name: str) -> np.array:
    img_src = Image.open(path.join('pictures_src', image_name)).convert('L')
    return np.array(img_src)

def integral_image(img: np.array) -> np.array:
    integral_img = np.zeros(shape=img.shape)
    integral_img[0, 0] = img[0, 0]

    for x in range(1, img.shape[0]):
        integral_img[x, 0] = img[x, 0] + integral_img[x - 1, 0]

    for y in range(1, img.shape[1]):
        integral_img[0, y] = img[0, y] + integral_img[0, y - 1]

    for x in range(1, img.shape[0]):
        for y in range(1, img.shape[1]):
            integral_img[x, y] = img[x, y] \
                                 - integral_img[x - 1, y - 1] \
                                 + integral_img[x - 1, y] \
                                 + integral_img[x, y - 1]

    return integral_img

def culculate_mean(integral_image: np.array, x: int, y: int, frame_size):
    square = frame_size**2
    s = sum_in_frame(integral_image, x, y, frame_size)
    return s // square

def sum_in_frame(integral_img: np.array, x: int, y: int, frame_size: int):
    len = integral_img.shape[1] - 1
    hight = integral_img.shape[0] - 1

    half_frame = frame_size // 2
    above = y - half_frame - 1
    low = y + half_frame
    left = x - half_frame - 1
    right = x + half_frame

    A = integral_img[max(above, 0), max(left, 0)]
    B = integral_img[max(0, above), min(len, right)]
    C = integral_img[min(hight, low), max(left, 0)]
    D = integral_img[min(hight, low), min(right, len)]

    if max(left + 1, 0) == 0 and max(above + 1, 0) == 0:
        return D
    elif max(left + 1, 0) == 0:
        return D - B
    elif max(above + 1, 0) == 0:
        return D - C

    return D - C - B + A

def semitone(img):
    return (0.3 * img[:, :, 0] + 0.59 * img[:, :, 1] + 0.11 *
            img[:, :, 2]).astype(np.uint8)

def bernsen_threshold(image: np.array, frame_size: int = 10, thres: int = 15):
    if len(image.shape) == 3:
        image = semitone(image)

    res_img = np.empty_like(image)
    integral_img = integral_image(image)

    for x in range(image.shape[0]):  
        for y in range(image.shape[1]):  
            mean = culculate_mean(integral_img, y, x, frame_size)
            if mean == 0:
                res_img[x, y] = 0
                continue

            p = image[x, y]
            ratio = 100 * p / mean

            if ratio < 100:
                if (100 - ratio) > thres:
                    res_img[x, y] = 0
                else:
                    res_img[x, y] = 255
            else:
                res_img[x, y] = 255

    return res_img

operators = {
    'x': np.array(
        [[-1, -2, -1],
         [0, 0, 0],
         [1, 2, 1]]
    ),

    'y': np.array(
        [[-1, 0, 1],
         [-2, 0, 2],
         [-1, 0, 1]]
    )
}

def get_frame(img: np.array, x: int, y: int) -> np.array:
    above = x - 1
    low = x + 2
    left = y - 1
    right = y + 2

    return img[above: low, left: right]

def apply_operator(frame: np.array, direction: str):
    frame = frame.astype(np.int32)

    if direction == 'x':
        return np.sum(operators['x'] * frame)
    elif direction == 'y':
        return np.sum(operators['y'] * frame)
    elif direction == 'g':
        return abs(np.sum(operators['x'] * frame)) + abs(np.sum(operators['y'] * frame))
    elif direction == 'b':
        return np.sqrt(np.sum((operators['x'] * frame) ** 2) + np.sum((operators['y'] * frame) ** 2))
    else:
        raise ValueError("Unsupported direction")

def sobel_operator(img: np.array, direction: str):
    new_img = np.zeros_like(img, dtype=np.float64)
    x, y = 1, 1

    while x < img.shape[0] - 1:
        if x % 2 == 0:
            while y + 1 < img.shape[1] - 1:
                frame = get_frame(img, x, y)
                new_img[x, y] = apply_operator(frame, direction)
                y += 1

        else:
            while y - 1 > 1:
                frame = get_frame(img, x, y)
                new_img[x, y] = apply_operator(frame, direction)
                y -= 1

        x += 1

    new_img = new_img / np.max(new_img) * 255

    if direction == 'b':
        print("Введите порог t:")
        t = int(input())
        return simple_bin(new_img, t).astype(np.uint8)
    elif direction == 'x' or direction == 'y' or direction == 'g':
        return new_img.astype(np.uint8)
    else:
        raise ValueError("Unsupported direction")

def simple_bin(img: np.array, thres: int = 40):
    res_img = np.empty_like(img)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            res_img[x, y] = 255 if img[x, y] > thres else 0
    return res_img

if __name__ == '__main__':
    images = {
        'Chess': 'chess_semitone.bmp',
        'Anime face': 'tsukyo_semitone.bmp',
        'House': 'house_semitone.bmp',
        'Face': 'face_semitone.bmp'
    }

    operations = {
        'Градиентная матрица G_x': 'x',
        'Градиентная матрица G_y': 'y',
        'Градиентная матрица G': 'g',
        'Бинаризованная градиентная матрица G_b': 'b',
    }

    selected_image = 'chess_semitone.bmp'  

    img = image_to_np_array(selected_image)

    print("Исходное изображение:")
    Image.fromarray(img, 'L').show()

    for operation_name, direction in operations.items():
        print("Результат применения операции:", operation_name)
        result = sobel_operator(img, direction)
        Image.fromarray(result, 'L').show()

    print("Бинаризованное изображение:")
    thresholded_img = sobel_operator(img, 'b')
    Image.fromarray(thresholded_img, 'L').show()


Исходное изображение:
Результат применения операции: Градиентная матрица G_x
Результат применения операции: Градиентная матрица G_y
Результат применения операции: Градиентная матрица G
Результат применения операции: Бинаризованная градиентная матрица G_b
Введите порог t:


 200


Бинаризованное изображение:
Введите порог t:
